In [ ]:
import numpy as np
import wandb
from matplotlib import pyplot as plt
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
import re
import plotly.express as px
import pandas as pd

In [ ]:
populate_plt_settings(plt)

In [ ]:
api = wandb.Api()

# runs = {
#     'Training set': api.run('kennychufk/alluvion-rl/2amh43ojAug-val'),
#     'Nephroid': api.run('kennychufk/alluvion-rl/2amh43ojAug-nephroid'),
#     'Bidirectional': api.run('kennychufk/alluvion-rl/2amh43ojAuga-bidir-circles'),
#     'Epitrochoid': api.run('kennychufk/alluvion-rl/2amh43ojAug-interesting-loop')
# }


runs = {
    'Training set': api.run('kennychufk/alluvion-rl/2v7m4mucAug-star-epicycloid'),
    'Nephroid': api.run('kennychufk/alluvion-rl/2v7m4mucAug-nephroid'),
    'Bidirectional': api.run('kennychufk/alluvion-rl/2v7m4mucAug-bidir-circles'),
    'Epitrochoid': api.run('kennychufk/alluvion-rl/2v7m4mucAug-interesting-loop'),
    'Diagonal': api.run('kennychufk/alluvion-rl/2v7m4mucAug-diagonal4')
}

In [ ]:
display_names_asc = [ '\emph{Stars \& Trefoiloids}', '\emph{Nephroids}', '\emph{Alternating loops}', '\emph{Epitrochoids}', '\emph{Diagonal oscillations}']
display_names = {}
for i, key in enumerate(runs):
    display_names[key]  = display_names_asc[i]

In [ ]:
histories = {}

for run_name in runs:
    run = runs[run_name]
    histories[run_name] = run.scan_history(keys=None,
                           page_size=1000,
                           min_step=None,
                           max_step=None)

In [ ]:
metric_name = 'eulerian_masked'
# metric_name = 'eulerian'

overall_name = f'overall-{metric_name}'


step_interval = 50
num_steps_per_sequence = 2000
num_artifacts = {}
overall_errors = {}
step_numbers = {}

for run_name in runs:
    num_artifacts_tmp = 0
    for row in histories[run_name]:
        num_artifacts_tmp+=((row['_step']+1)%step_interval==0)
    num_artifacts[run_name]=num_artifacts_tmp
    
    step_numbers[run_name] = (np.arange(num_artifacts_tmp)+1)*step_interval*num_steps_per_sequence

    overall_errors[run_name] = np.zeros(num_artifacts_tmp)

    artifact_id = 0
    for row_id, row in enumerate(histories[run_name]):
        if (row['_step']+1)%50!=0:
            continue
        for key in row:
            if key == overall_name:
                overall_errors[run_name][artifact_id] = row[key]
        artifact_id+=1

In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
num_rows = 1
num_cols = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width(), ratio=(np.sqrt(5)-1)*0.5))


lines = [ax.plot(step_numbers[run_name], overall_errors[run_name]/np.max(overall_errors[run_name]), label=display_names[run_name], zorder=2)[0] for run_name in ['Training set', 'Diagonal', 'Nephroid', 'Bidirectional', 'Epitrochoid']]

    
ax.set_xlabel('Steps')
ax.set_ylabel(r'Normalized Eulerian error')
ax.set_xlim(0,6000000)
ax.set_ylim(0.65,1)
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', color='#DDDDDD', linewidth=0.5)

legend_left = ax.legend(handles=[lines[3], lines[4]], loc='upper center', bbox_to_anchor=(0.44, 0.98), ncol=1, frameon=True, handlelength=0.8, handleheight=0.8, labelspacing=0.3, borderpad=0, handletextpad=0.2,borderaxespad=0,columnspacing=1.0, edgecolor='None')

ax.add_artist(legend_left)
legend_right = ax.legend(handles=[lines[0], lines[1], lines[2]], loc='upper center', bbox_to_anchor=(0.79, 0.98), ncol=1, frameon=True, handlelength=0.8, handleheight=0.8, labelspacing=0.3, borderpad=0, handletextpad=0.2,borderaxespad=0,columnspacing=1.0, edgecolor='None')
legend_left.set_zorder(1)
legend_right.set_zorder(1)


fig.tight_layout(pad=0.05) # should set tight_layout before add_axes()
fig.savefig('learning-curve-generality.pdf')

In [ ]:
step_numbers['Training set']